# Models Comparison

## 1. Test Dataset Preparation

### 1. 1 Load Dataset Metadata

In [1]:
import os
from pathlib import Path
import scipy

# Set path to data directory
data_dir = Path("/kaggle/input/102-flowers")

# Set path to label files & split files
label_file = data_dir / "imagelabels.mat"
split_file = data_dir / "setid.mat" 

In [2]:

labels_data = scipy.io.loadmat(label_file)
splits_data = scipy.io.loadmat(split_file)

labels_data, splits_data

({'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Thu Feb 19 15:43:33 2009',
  '__version__': '1.0',
  '__globals__': [],
  'labels': array([[77, 77, 77, ..., 62, 62, 62]], dtype=uint8)},
 {'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Thu Feb 19 17:38:58 2009',
  '__version__': '1.0',
  '__globals__': [],
  'trnid': array([[6765, 6755, 6768, ..., 8026, 8036, 8041]], dtype=uint16),
  'valid': array([[6773, 6767, 6739, ..., 8028, 8008, 8030]], dtype=uint16),
  'tstid': array([[6734, 6735, 6737, ..., 8044, 8045, 8047]], dtype=uint16)})

### 1.2 Extract label and split indices

In [3]:

labels = labels_data['labels'].flatten()
train_indices = splits_data['trnid'].flatten()
test_indices = splits_data['tstid'].flatten()
val_indices = splits_data['valid'].flatten()

print(f'First five labels: {labels[:5]}')
print(f'Min value of train_indices: {train_indices.min()}')
print(f'Min value of test_indices: {test_indices.min()}')
print(f'Min value of val_indices: {val_indices.min()}')
print()
print(f'Length of train_indices: {len(train_indices)}')
print(f'Length of test_indices: {len(test_indices)}')
print(f'Length of val_indices: {len(val_indices)}')


First five labels: [77 77 77 77 77]
Min value of train_indices: 28
Min value of test_indices: 1
Min value of val_indices: 17

Length of train_indices: 1020
Length of test_indices: 6149
Length of val_indices: 1020


### 1.3 Map indices to Image Path

In [4]:

# Load and sort all the image files
data_path = data_dir / "images/images"
images_files = sorted(list(data_path.glob("*.jpg")))

images_files[:5]

[PosixPath('/kaggle/input/102-flowers/images/images/image_00001.jpg'),
 PosixPath('/kaggle/input/102-flowers/images/images/image_00002.jpg'),
 PosixPath('/kaggle/input/102-flowers/images/images/image_00003.jpg'),
 PosixPath('/kaggle/input/102-flowers/images/images/image_00004.jpg'),
 PosixPath('/kaggle/input/102-flowers/images/images/image_00005.jpg')]

In [5]:
# Create lists of image paths and corresponding labels for the training set.
# i-1 adjustment is done for matching the indices from matlab to python's zero based system
train_images = [images_files[i-1] for i in train_indices]  
train_labels = [labels[i-1] for i in train_indices]

test_images = [images_files[i-1] for i in test_indices]
test_labels = [labels[i-1] for i in test_indices]


val_images = [images_files[i-1] for i in val_indices]
val_labels = [labels[i-1] for i in val_indices]

# Checking the data
print(f'Min train_images value: {min(train_images)}')
print(f'Min test_values: {min(test_images)}')
print(f'Min val_values: {min(val_images)}')
print()
print(f'Length of train_images: {len(train_images)}')
print(f'Length of test_images {len(test_images)}')
print(f'Length of val_images: {len(val_images)}')

Min train_images value: /kaggle/input/102-flowers/images/images/image_00028.jpg
Min test_values: /kaggle/input/102-flowers/images/images/image_00001.jpg
Min val_values: /kaggle/input/102-flowers/images/images/image_00017.jpg

Length of train_images: 1020
Length of test_images 6149
Length of val_images: 1020


## 2. Create Custom Dataset

### 2.1  Define Transformations

In [6]:

import torch
from torchvision import transforms
from torchvision import models

# Load pretrained weights for ResNet50
pretrained_weights = models.ResNet50_Weights.DEFAULT

# Define the common transforms for input data
common_transforms = pretrained_weights.transforms()
common_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [7]:
# Define test transformation
test_transform = transforms.Compose([
    transforms.Resize([232]),
    transforms.CenterCrop([224]),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print(test_transform)

Compose(
    Resize(size=[232], interpolation=bilinear, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


### 2.2 Create custom dataloader

In [8]:

device = 'cpu'
device

'cpu'

In [9]:

from torch.utils.data import Dataset
from PIL import Image
from collections import Counter

class CustomDataset(Dataset):
    def __init__(self, data_path, labels, transform=None):  # Make transform optional
        self.data_path = data_path
        self.labels = labels
        self.transform = transform
        
    def __getitem__(self, idx: int):
        img = Image.open(self.data_path[idx]) 
        img.convert('RGB')

        if self.transform:
            img = self.transform(img)  

        label = self.labels[idx] - 1  # Assuming labels are 1-indexed
        return img, label
        
    def __len__(self):
        return len(self.data_path)


test_dataset = CustomDataset(test_images, test_labels, test_transform)
print(f'Length of test_dataset {len(test_dataset)}')

Length of test_dataset 6149


### 2.3 Create Dataloaders

In [10]:

from torch.utils.data import DataLoader

BATCH_SIZE = 16

# Create the DataLoader for the testing dataset
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE) 


## 3. Helper Functions

In [11]:

import os
import psutil
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

def get_pytorch_model_size(model):
    total_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    return round(total_bytes / (1024 * 1024), 2)
    
def get_onnx_size(path):
    size_mb = os.path.getsize(path) / (1024 * 1024)
    return round(size_mb, 2)

def measure_memory():
    process = psutil.Process()
    return process.memory_info().rss / (1024 * 1024)  # MB

def evaluate_predictions(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)

    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="macro", zero_division=0
    )

    return {
        "accuracy": acc,
        "precision_macro": precision,
        "recall_macro": recall,
        "f1_macro": f1,
    }


## 4. Evaluate Models

### 4.1 Evaluate PyTorch Model

In [12]:

import torch
import torch.nn as nn
from torchvision import models
import time
from tqdm import tqdm
import tracemalloc


# Recreate architecture exactly as training
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

num_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Dropout(0.4664370979686941),   # Tuned dropout
    nn.Linear(num_features, 102)      # 102-class classifier
)

model = model.to(device)

# Load state_dict
state_dict = torch.load(
    "/kaggle/input/flower-pytorch/pytorch/default/1/best_flower_model.pth",
    map_location=device
)

model.load_state_dict(state_dict) 

# Set to eval
model.eval()

pytorch_model = model

y_true = []
y_pred = []

tracemalloc.start()

start = time.time()

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)

        outputs = pytorch_model(images)
        preds = outputs.argmax(1)

        y_true.extend(labels.numpy())
        y_pred.extend(preds.cpu().numpy())

torch_time = time.time() - start

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
torch_mem_used = peak / (1024 * 1024)

torch_size = get_pytorch_model_size(pytorch_model)
torch_metrics = evaluate_predictions(y_true, y_pred)

print(f"{torch_time}\n{torch_mem_used}\n{torch_size}\n{torch_metrics}")


100%|██████████| 385/385 [08:41<00:00,  1.35s/it]

521.4858448505402
2.0279531478881836
90.47
{'accuracy': 0.9139697511790535, 'precision_macro': 0.9033807248029921, 'recall_macro': 0.9296607032305559, 'f1_macro': 0.9103818595982978}


### 4.2 Evaluate ONNX Model

In [13]:
!pip install onnxruntime

In [14]:

import onnxruntime as ort
import numpy as np
import tracemalloc

ort_session = ort.InferenceSession(
    "/kaggle/input/flower-onnx/onnx/default/1/resnet50_flower.onnx",
    providers=["CPUExecutionProvider"]
)

input_name = ort_session.get_inputs()[0].name

y_true = []
y_pred = []

tracemalloc.start()
start = time.time()

for images, labels in tqdm(test_loader):
    inputs = images.numpy()  # dataloader gives tensors → convert to numpy

    outputs = ort_session.run(None, {input_name: inputs})
    preds = np.argmax(outputs[0], axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

onnx_time = time.time() - start

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
onnx_mem_used = peak / (1024 * 1024)

onnx_size = get_onnx_size("/kaggle/input/flower-onnx/onnx/default/1/resnet50_flower.onnx")

onnx_metrics = evaluate_predictions(y_true, y_pred)

print(f"{onnx_time}\n{onnx_mem_used}\n{onnx_size}\n{onnx_metrics}")


100%|██████████| 385/385 [04:59<00:00,  1.28it/s]

299.88049125671387
1.0969066619873047
90.4
{'accuracy': 0.9139697511790535, 'precision_macro': 0.9033807248029921, 'recall_macro': 0.9296607032305559, 'f1_macro': 0.9103818595982978}


## 5. Summary Table

In [15]:

import pandas as pd

results = pd.DataFrame([
    ["PyTorch", torch_metrics["accuracy"], torch_metrics["f1_macro"], torch_time, torch_size, torch_mem_used],
    ["ONNX", onnx_metrics["accuracy"], onnx_metrics["f1_macro"], onnx_time, onnx_size, onnx_mem_used],
],
columns=["Model", "Accuracy", "F1 (Macro)", "Inference Time (s)", "Model Size (MB)", "Memory (MB)"]
)

results


,Model,Accuracy,F1 (Macro),Inference Time (s),Model Size (MB),Memory (MB)
0,PyTorch,0.91397,0.910382,521.485845,90.47,2.027953
1,ONNX,0.91397,0.910382,299.880491,90.40,1.096907
